# Text classification with movie reivews
<hr>

이 notebook은 영화리뷰의 텍스트를 이용하여 긍정인지 부정인지를 분류한다. 이러한 긍, 부정 분류는 2중 분류 (binary, or two-class) 한 예시이고, 머신러닝 문제에서 넓게 적용할 수 있다.

사용될 데이터셋인 [IMDB dataset](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb)는:
- 50,000 영화 리뷰 포함하며
- training과 testing set들은 같은 각 set 내에서 같은 수의 긍정, 부정 리뷰를 담고있다.

여기서는 model을 구성하고 학습하기 하기 위한 high-level API인 `tf.keras`를 사용하며, `tf.keras`를 사용하여 text classification에 대해 더 심화된 가이드를 얻고 싶다면 [MLCC Text Classification Guide](https://developers.google.com/machine-learning/guides/text-classification/) 여기를 참고하라.

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

1.12.0


## Download the IMDB dataset
<hr>

IMDB dataset를 tensorflow로 다운받을 경우, 전처리가 되어있어 (단어들이 sequence 되어있음) 각 정수(각 정수는 dictionary에서 특정 단어를 표현)의 sequence로 변환되어있다.

다음 코드에서 IMDB dataset을 받아보자:

In [2]:
imdb = keras.datasets.imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

`num_words=1000`은 training set에서 자주 사용되는 단어 상위 10,000개를 추출하라는 argument이다. 이를 통해 데이터를 다루기 좋을 사이즈로 유지하기 위해서 드물게 사용되는 단어들은 제거됩니다.

## Explore the data
<hr>

data format을 유심히 보자. dataset이 전처리가 되어있기때문에, 각 예제는 영화 리뷰 단어들의 정수배열을 나타낸다. 각 label은 0 혹은 1인 정수를 가질 것이다. <br>*(0: negative review, 1: positive review)*

In [5]:
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


### convert the integers back to words

## Prepare the data
<hr>

## Build the model
<hr>

### Hidden units

### Loss function and optimizer

## Create a validation set
<hr>

## Train the model
<hr>

## Evaluate the model
<hr>

## Create a graph of accuracy and loss over time
<hr>